## Encoding input data sequence

In [9]:
import numpy as np
from scipy.optimize import fsolve


def lowbit(x):
    return x & -x


def hamming(s) -> str:
    ctrl = int(np.round(fsolve(lambda x: 2 ** x - 1 - x - len(s), np.log2(len(s) + 10))[0]))
    assert len(s) + ctrl + 1 == 2 ** ctrl
    cnt = 0
    code = [-1]
    for i in range(1, len(s) + ctrl + 1):
        if i == lowbit(i):
            code.append(0)
            cnt += 1
        else:
            code.append(int(s[cnt - i]))
    for i in range(1, len(s) + ctrl + 1):
        if i != lowbit(i):
            for k in range(ctrl):
                if i & (1 << k):
                    code[1 << k] ^= code[i]
    return ''.join(map(str, code[-1:0:-1]))


if __name__ == "__main__":
    print(hamming("0"), hamming("1010"), hamming("10110111010"), hamming("10" * 12))

000 1010010 101101111011011 1010101010101010010101011010010


## 解释

代码可对长度为 $2 ^ t - t - 1, t \in \mathbb{N}, t \geq 2$ 的数据编码为汉明码。

### 获取二进制最高位

从树状数组那里搬过来的。

```py
def lowbit(x):
    return x & -x
```

### 求解控制位长度

```py
    ctrl = int(np.round(fsolve(lambda x: 2 ** x - 1 - x - len(s), np.log2(len(s) + 10))[0]))
    assert len(s) + ctrl + 1 == 2 ** ctrl
```

这里猜测的初值大于 $\log_2 L$，使得求解大于 0，随后验证输入长度是否满足汉明码要求。

### 初始状态

填充汉明码，如果是独位二进制数，则为校验位，故填充 $0$，否则填充 `s[cnt - i]`，其中 `cnt` 表示已经‘
有几个位置被校验位挤占了。

例如 $i = 3$，前两个都是校验位，则将 `s[2 - 3]` 即 `s[-1]`
填进来（`s` 最后一位填充到汉明码数据位第一位）。

```py
    cnt = 0
    code = [-1]
    for i in range(1, len(s) + ctrl + 1):
        if i == lowbit(i): # 等于最高位自然就是独位了
            code.append(0)
            cnt += 1
        else:
            code.append(int(s[cnt - i]))
```

### 非独位

```py
    for i in range(1, len(s) + ctrl + 1):
        if i != lowbit(i):
            for k in range(ctrl):
                if i & (1 << k):
                    code[1 << k] ^= code[i]
    return ''.join(map(str, code[-1:0:-1]))
```

对于数据位，枚举二进制每一位，异或贡献到对应校验位。最后反向输出，因为求解过程中先反了一次下标。

总复杂度 $O(L \log L)$

---

### 其他

如果长度不满足 $2^t - t - 1$ 可考虑补零，然而下面 correction 也没办法优雅补零。

## Error detection and correction

In [15]:
import numpy as np
def correction(s):
    ctrl = int(np.log2(len(s)) + 1)
    li = list(map(int, s))
    x = [0] * ctrl # 1 << k xor
    for i, c in enumerate(reversed(li)):
        idx = i + 1
        for k in range(ctrl):
            if idx & (1 << k):
                x[k] ^= c
    w = sum(
        map(
            lambda x: x[1] << x[0], enumerate(x)
        ) 
    )
    if not w:
        print('The message is correct.\n')
    else:
        li[len(s) - w] ^= 1
        print(f"{w} is wrong.\nCorrection: {''.join(map(str, li))}\n")


if __name__ == "__main__":
    correction("100")
    correction("000")
    correction("1010110")
    correction("101101011011011")
    correction("0010101010101010010101011010010")

3 is wrong.
Correction: 000

The message is correct.

3 is wrong.
Correction: 1010010

9 is wrong.
Correction: 101101111011011

31 is wrong.
Correction: 1010101010101010010101011010010



## 解释

代码可纠错长度为 $2 ^ t - 1, t \in \mathbb{N}, t >= 1$ 的汉明码，最多有一个错位。

### 获取对应的控制位长度

```
    ctrl = int(np.log2(len(s)) + 1)
```

### 求解每个二进制位的偶校验和

对于每个 $k \in [1, \texttt{ctrl}]$，求解二进制第 $k$ 位为 $1$ 的所有位的数据异或和.

```py
    li = list(map(int, s))
    x = [0] * ctrl # 1 << k xor
    for i, c in enumerate(reversed(li)):
        idx = i + 1
        for k in range(ctrl):
            if idx & (1 << k):
                x[k] ^= c
```

### 累加异常数据位

例如，`enumerate(x)` 形如 `( (0, 1), (1, 0), (2, 1) )`，表示：

- 二进制第 0 位的异或和有异常
- 二进制第 1 位的异或和没有异常
- 二进制第 0 位的异或和有异常

则求解 $(1 << 0) + (0 << 1) + (1 << 2)$ 即求出异常位。

```py
    w = sum(
        map(
            lambda x: x[1] << x[0], enumerate(x)
        ) 
    )
```

### 判断和修正

```py
    if not w:
        print('The message is correct.\n')
    else:
        li[len(s) - w] ^= 1
        print(f"{w} is wrong.\nCorrection: {''.join(map(str, li))}\n")
```

有异常则修正反向位置。